# Imports

In [1]:
import numpy as np

In [2]:
import os.path as osp
import numpy as np
from tqdm import tqdm
from utils import pickle_save, pickle_load
from pprint import pprint
from utils.data.delf import datum_io
from copy import deepcopy

In [3]:
import sacred
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from numpy import linalg as LA
from utils import pickle_load, pickle_save
from utils.revisited import compute_metrics
from utils.data.delf import datum_io

In [4]:
help(compute_metrics)

Help on function compute_metrics in module utils.revisited:

compute_metrics(dataset, ranks, gnd, sizes=[], kappas=[1, 5, 10], set_name=None, max_sequence_len=None)



In [5]:
ex = sacred.Experiment('Prepare Top-K (VIQUAE FOR RTT)', interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [21]:
feature_name = 'imagenet_r50'
set_name = 'tuto'
gnd_name = 'gnd_'+ set_name+'.pkl'

In [22]:
dataset_name = 'viquae_for_rrt'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [8]:
training = False
use_aqe = False
aqe_params = {'k': 2, 'alpha': 0.3}

save_nn_inds = True
prefixed = 'humans'

In [9]:
query_file     = prefixed + '_'+ set_name+'_query.txt'
gallery_file   = prefixed + '_'+ set_name+'_gallery.txt'
selection_file = prefixed + '_'+ set_name+'_selection.txt'

In [10]:
with open(osp.join(data_dir, query_file)) as fid:
    query_lines   = fid.read().splitlines()

In [11]:
len(query_lines)

63

In [23]:
query_feats = []
for i in tqdm(range(len(query_lines))):
    name = osp.splitext(osp.basename(query_lines[i].split(';;')[0]))[0]
    path = osp.join(data_dir, feature_name, name + '.imagenet')
    query_feats.append(datum_io.ReadFromFile(path))

100%|██████████| 63/63 [00:00<00:00, 682.43it/s]


In [24]:
query_feats = np.stack(query_feats, axis=0)
query_feats = query_feats / LA.norm(query_feats, axis=-1)[:, None]

In [25]:
query_feats.shape

(63, 2048)

In [26]:
with open(osp.join(data_dir, selection_file)) as fid:
    selection_lines = fid.read().splitlines()

In [27]:
np.stack(selection_lines, axis=0).shape

(3089,)

In [28]:
osp.basename(selection_lines[i].split(';;')[0])

'512px-Ruby_Dee_speaking.jpg'

In [30]:
index_feats = []
for i in tqdm(range(len(selection_lines))):
    name = osp.splitext(osp.basename(selection_lines[i].split(';;')[0]))[0]
    path = osp.join(data_dir, feature_name, name+'.delg_global')
    print(type(datum_io.ReadFromFile(path)))
    index_feats.append(datum_io.ReadFromFile(path))

  0%|          | 0/3089 [00:00<?, ?it/s]


NotFoundError: /mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/imagenet_r50/512px-Jack_Ruby-1.delg_global; No such file or directory

In [18]:
selection_index_feats = np.zeros((query_feats.shape[0], 100, query_feats.shape[1]))
selection_index_feats.shape

(100, 100, 2048)

In [19]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))

In [20]:
selection_index_sizes = [len(gnd_data['simlist'][i]) for i in range(len(gnd_data['simlist']))]
np.sum(selection_index_sizes)

4838

In [21]:
size = 0
counter = 0
for i in range(selection_index_feats.shape[0]):
    for j in range(selection_index_feats.shape[1]):
        if j < selection_index_sizes[i]:
            selection_index_feats[i][j] = index_feats[counter]
            counter += 1

In [22]:
selection_index_feats.shape

(100, 100, 2048)

In [23]:
for i in range(selection_index_feats.shape[0]):
    selection_index_feats[i] = selection_index_feats[i]/LA.norm(selection_index_feats[i], axis=-1)[:,None]

/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [24]:
sims = []
for i in range(len(selection_index_feats)):
    index_feats = np.stack(selection_index_feats[i], axis=0)
    sims.append(np.matmul(query_feats[i], index_feats.T))

In [25]:
sims = np.stack(sims, axis=0)
sims.shape

(100, 100)

In [26]:
sims

array([[-0.04103276,  0.16977126,  0.18141674, ...,         nan,
                nan,         nan],
       [ 0.33237307,  0.3944457 ,  0.42619404, ...,         nan,
                nan,         nan],
       [ 0.33237307,  0.30754114,  0.44644287, ...,         nan,
                nan,         nan],
       ...,
       [ 0.19000561, -0.01076683,  0.03394178, ...,         nan,
                nan,         nan],
       [ 0.04822452,  0.24529736,  0.21718944, ...,         nan,
                nan,         nan],
       [ 0.03589981,  0.16352484,  0.21618424, ...,         nan,
                nan,         nan]])

In [27]:
np.count_nonzero(np.isnan(sims)), np.count_nonzero(~np.isnan(sims))

(5162, 4838)

In [28]:
if use_aqe:
    alpha = aqe_params['alpha']
    nn_inds = np.argsort(-sims, -1)
    query_aug = deepcopy(query_feats)
    for i in range(len(query_feats)):
        new_q = [query_feats[i]]
        for j in range(aqe_params['k']):
            nn_id = nn_inds[i, j]
            weight = sims[i, nn_id] ** aqe_params['alpha']
            new_q.append(weight * index_feats[nn_id])
        new_q = np.stack(new_q, 0)
        new_q = np.mean(new_q, axis=0)
        query_aug[i] = new_q/LA.norm(new_q, axis=-1)
    sims = np.matmul(query_aug, index_feats.T)

In [29]:
selection_index_feats[0].shape

(100, 2048)

In [30]:
nn_inds = np.argsort(-sims, -1)
nn_dists = deepcopy(sims)
for i in range(query_feats.shape[0]):
    index_feats = selection_index_feats[i]
    for j in range(index_feats.shape[0]):
        nn_dists[i, j] = sims[i, nn_inds[i, j]]

In [31]:
nn_inds.shape

(100, 100)

In [32]:
nn_dists

array([[0.53691091, 0.48823032, 0.3786902 , ...,        nan,        nan,
               nan],
       [0.50516481, 0.46841804, 0.44644287, ...,        nan,        nan,
               nan],
       [0.50516481, 0.46841804, 0.45547155, ...,        nan,        nan,
               nan],
       ...,
       [0.24880777, 0.19662176, 0.19087987, ...,        nan,        nan,
               nan],
       [0.27114646, 0.24529736, 0.21771399, ...,        nan,        nan,
               nan],
       [0.26374199, 0.24880777, 0.22943023, ...,        nan,        nan,
               nan]])

In [33]:
for i in range(len(selection_index_sizes)):
    size = selection_index_sizes[i]
    if max(nn_inds[i][:size]) > size:
        print('Ewwwwwwwwwwwww')

In [34]:
if save_nn_inds:
    if use_aqe:
        output_file = set_name +'_aqe_nn_inds_%s.pkl' % feature_name
    else:
        output_file = set_name + '_nn_inds_%s.pkl' % feature_name

    if training:
        output_file = 'training_'+ output_file

    output_path = osp.join(data_dir, output_file)
    pickle_save(output_path, nn_inds)

In [35]:
from utils.revisited import compute_metrics

In [36]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))
compute_metrics('viquae', nn_inds.T, gnd_data['gnd'], selection_index_sizes, kappas=[1,5,10])

{'map': 30.28, 'mrr': 41.97, 'precision': 13.64, 'hit_rate': 83.0, 'recall': 82.5, 'map@1': 10.25, 'mrr@1': 29.0, 'precision@1': 29.0, 'hit_rate@1': 29.0, 'recall@1': 10.25, 'map@5': 17.72, 'mrr@5': 39.85, 'precision@5': 18.8, 'hit_rate@5': 56.0, 'recall@5': 26.02, 'map@10': 21.21, 'mrr@10': 40.93, 'precision@10': 16.6, 'hit_rate@10': 63.0, 'recall@10': 35.0}


{'map': 30.28,
 'mrr': 41.97,
 'precision': 13.64,
 'hit_rate': 83.0,
 'recall': 82.5,
 'map@1': 10.25,
 'mrr@1': 29.0,
 'precision@1': 29.0,
 'hit_rate@1': 29.0,
 'recall@1': 10.25,
 'map@5': 17.72,
 'mrr@5': 39.85,
 'precision@5': 18.8,
 'hit_rate@5': 56.0,
 'recall@5': 26.02,
 'map@10': 21.21,
 'mrr@10': 40.93,
 'precision@10': 16.6,
 'hit_rate@10': 63.0,
 'recall@10': 35.0}

In [37]:
from utils.revisited import compute_ap, compute_map
from ranx import Qrels, Run, evaluate

In [46]:
gnd[i]['provenance_entity']

True

In [47]:
def compute_metrics(dataset, ranks, gnd, sizes=[], kappas=[1, 5, 10]):
    # old evaluation protocol
    if dataset.startswith('classic'):
        map, aps, _, _ = compute_map(ranks, gnd)
        out = {'map': np.around(map*100, decimals=3)}
        print('>> {}: mAP {:.2f}'.format(dataset, out['map']))

    # new evaluation protocol for viquae dataset
    elif dataset.startswith('viquae'):
        metrics = ["map", "mrr", "precision", "hit_rate", "recall"]
        m_list = [metric for metric in metrics]

        for i in range(len(kappas)):
            m_list.extend([metric+'@'+str(kappas[i]) for metric in metrics])
        
        qrels_dict = {}
        run_dict = {}
        
        for i in range(ranks.T.shape[0]):
            size = sizes[i]
            q_str = "q_"+str(int(i))
            if gnd[i]['provenance_entity']:
                ok_inds = gnd[i]['r_hard']
            else:
                ok_inds = np.concatenate([gnd[i]['r_easy'], gnd[i]['r_hard']])
            
            if len(ok_inds) == 0:
                qrels_dict[q_str] = {"DUMMY_RUN": 0}
            else:
                qrels_dict[q_str] = dict([('d_' + str(int(key)), 1) for key in ok_inds])
            
            run_dict[q_str]   = dict([('d_' + str(int(key)), 1) for key in ranks[:size,i]])

        qrels = Qrels(qrels_dict)
        run = Run(run_dict)
        out = evaluate(qrels, run, m_list)
        for key, value in out.items():
            out[key] = np.around(value*100, decimals=2)
    print(out)
    
    return out

In [48]:
kappas=[1,5,10]
ranks, gnd, sizes = nn_inds.T, gnd_data['gnd'], selection_index_sizes

metrics = ["map", "mrr", "precision", "hit_rate", "recall"]
m_list = [metric for metric in metrics]

for i in range(len(kappas)):
    m_list.extend([metric+'@'+str(kappas[i]) for metric in metrics])

qrels_dict = {}
run_dict = {}

for i in range(ranks.T.shape[0]):
    size = sizes[i]
    q_str = "q_"+str(int(i))
    ok_inds = np.concatenate([gnd[i]['r_easy'], gnd[i]['r_hard']])
    
    if len(ok_inds) == 0:
        qrels_dict[q_str] = {"DUMMY_RUN": 0}
    else:
        qrels_dict[q_str] = dict([('d_' + str(int(key)), 1) for key in ok_inds])
    run_dict[q_str]   = dict([('d_' + str(int(key)), 1) for key in ranks[:size,i]])

qrels = Qrels(qrels_dict)
run = Run(run_dict)
out = evaluate(qrels, run, m_list)
for key, value in out.items():
    out[key] = np.around(value*100, decimals=2)
print(out)

{'map': 30.28, 'mrr': 41.97, 'precision': 13.64, 'hit_rate': 83.0, 'recall': 82.5, 'map@1': 10.25, 'mrr@1': 29.0, 'precision@1': 29.0, 'hit_rate@1': 29.0, 'recall@1': 10.25, 'map@5': 17.72, 'mrr@5': 39.85, 'precision@5': 18.8, 'hit_rate@5': 56.0, 'recall@5': 26.02, 'map@10': 21.21, 'mrr@10': 40.93, 'precision@10': 16.6, 'hit_rate@10': 63.0, 'recall@10': 35.0}


In [44]:
run_dict
[k for k,v in run_dict.items() if v == {}]

[]

In [45]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))
compute_metrics('viquae', nn_inds.T, gnd_data['gnd'], selection_index_sizes, kappas=[1,5,10])

{'map': 30.28, 'mrr': 41.97, 'precision': 13.64, 'hit_rate': 83.0, 'recall': 82.5, 'map@1': 10.25, 'mrr@1': 29.0, 'precision@1': 29.0, 'hit_rate@1': 29.0, 'recall@1': 10.25, 'map@5': 17.72, 'mrr@5': 39.85, 'precision@5': 18.8, 'hit_rate@5': 56.0, 'recall@5': 26.02, 'map@10': 21.21, 'mrr@10': 40.93, 'precision@10': 16.6, 'hit_rate@10': 63.0, 'recall@10': 35.0}


{'map': 30.28,
 'mrr': 41.97,
 'precision': 13.64,
 'hit_rate': 83.0,
 'recall': 82.5,
 'map@1': 10.25,
 'mrr@1': 29.0,
 'precision@1': 29.0,
 'hit_rate@1': 29.0,
 'recall@1': 10.25,
 'map@5': 17.72,
 'mrr@5': 39.85,
 'precision@5': 18.8,
 'hit_rate@5': 56.0,
 'recall@5': 26.02,
 'map@10': 21.21,
 'mrr@10': 40.93,
 'precision@10': 16.6,
 'hit_rate@10': 63.0,
 'recall@10': 35.0}